In [2]:
%load_ext autoreload
%autoreload 1

%aimport basics
%aimport nn_config
%aimport enc_dec


from basics import *
from nn_config import *
from enc_dec import *
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
translating es to en
callhome es-en word level configuration
vocab size, en=51, fr=51


IndentationError: expected an indented block (enc_dec.py, line 62)

In [ ]:
xp = cuda.cupy if gpuid >= 0 else np

In [ ]:
text_data = pickle.load(open(text_data_dict, "rb"))

In [ ]:
if gpuid >= 0:
    # print("here")
    cuda.get_device(gpuid).use()

In [ ]:
model = SpeechEncoderDecoder(SPEECH_DIM, vocab_size_en, num_layers_enc, num_layers_dec,
                               hidden_units, gpuid, attn=use_attn)
if gpuid >= 0:
    cuda.get_device(gpuid).use()
    model.to_gpu()

In [ ]:
log_train_fil_name, text_fname, dev_fname, test_fname

In [ ]:
%aimport nmt_trials
from nmt_trials import *

In [ ]:
buckets, bucket_lengths = populate_buckets()

In [ ]:
25*32, 24*35

In [ ]:
_, en, sf = get_data_item('053.181')
xp.isnan(xp.sum(sf))

In [ ]:
buck_indx = 23
i = 0
batch_size = 50

pad_size_speech = (buck_indx+1) * SPEECH_BUCKET_WIDTH
pad_size_en = min(bucket_lengths[buck_indx][3], MAX_EN_LEN)

src_lim = pad_size_speech
tar_lim = pad_size_en

sp_files_in_batch = [t[0] for t in buckets[buck_indx][i:i+batch_size]]

# get the next batch of data
batch_data = []
for sp_fil in sp_files_in_batch:
    _, en_ids, speech_feat = get_data_item(sp_fil, cat="train")
    # print(speech_feat.shape, len(en_ids))
    batch_data.append((speech_feat[:pad_size_speech], en_ids[:pad_size_en]))

# compute loss
# loss = model.encode_decode_train_batch(batch_data, pad_size_speech, pad_size_en, train=True)

In [ ]:
batch_size = len(batch_data)
in_shape_r, in_shape_c = batch_data[0][0].shape

fwd_encoder_batch = xp.full((batch_size, pad_size_speech, in_shape_c), PAD_ID, dtype=xp.float32)
rev_encoder_batch = xp.full((batch_size, pad_size_speech, in_shape_c), PAD_ID, dtype=xp.float32)
decoder_batch = xp.full((batch_size, pad_size_en+2), PAD_ID, dtype=xp.int32)

In [ ]:
for i, (src, tar) in enumerate(batch_data):
    fwd_encoder_batch[i] = model.pad_array(src, src_lim)
    rev_encoder_batch[i] = model.pad_array(xp.flip(src, axis=0), src_lim)

    tar_data = [GO_ID] + tar + [EOS_ID]
    decoder_batch[i] = model.pad_list(tar_data, tar_lim+2, at_start=False)

In [ ]:
train=True

In [ ]:
fwd_encoder_batch = xp.swapaxes(fwd_encoder_batch, 0,1)
rev_encoder_batch = xp.swapaxes(rev_encoder_batch, 0,1)
decoder_batch = xp.swapaxes(decoder_batch, 0,1)

In [ ]:
L_FWD_STATES = model.encode_speech_batch_lstm(fwd_encoder_batch, model.lstm_enc, train=True)

In [ ]:
L_REV_STATES = model.encode_speech_batch_lstm(rev_encoder_batch, model.lstm_rev_enc, train)
# reverse the states to align them with forward encoder
L_REV_STATES = xp.flip(L_REV_STATES, axis=0)

In [ ]:
return_shape = L_FWD_STATES.shape
model.enc_states = F.concat((L_FWD_STATES, L_REV_STATES), axis=2)
model.enc_states = F.swapaxes(model.enc_states, 0, 1)

In [ ]:
model.encode_decode_train_batch(batch_data, pad_size_speech, pad_size_en, train=train)

In [ ]:
L_FWD_STATES.shape

In [ ]:
pad_size_speech / 8

In [ ]:
in_size, batch_size, in_dim = fwd_encoder_batch.shape

In [ ]:
print(in_size, batch_size, in_dim)

In [ ]:
speech_feats = xp.load(os.path.join(speech_dir, "train.npz"))

In [ ]:
haha = speech_feats["041.001"]

In [ ]:
haha.shape

In [ ]:
haha = xp.array([[2,-2], [1,-1], [3,-3]], dtype=xp.float32)

In [ ]:
haha

In [ ]:
meh = F.expand_dims(haha.swapaxes(0,1), axis=0)

In [ ]:
meh.shape

In [7]:
def cnn_out_size(i, p, k):
    o = (i-k) + 2*p + 1
    print("cnn out = {0:d}".format(o))

In [ ]:
k_width = 2
pad = k_width // 2
# pad = k_width - 1
pad = 1

In [ ]:
if gpuid >= 0:
    # print("here")
    cuda.get_device(gpuid).use()
c1 = L.ConvolutionND(ndim=1, in_channels=SPEECH_DIM, out_channels=1, ksize=k_width, pad=pad).to_gpu()

In [ ]:
c1.W.shape

In [ ]:
i = meh.shape[2]
out_shape = (i-k_width) + 2*pad + 1
print(i, out_shape)

In [ ]:
%%timeit -n1 -r2
ha = c1(meh)
print(ha.shape)
print(ha.data)

### test with small arrays

In [75]:
mini_test = xp.array([[2,-2,-5], [1,-1,-10], [3,-3,-9], [4,-4,-12]], dtype=xp.float32)

In [76]:
mini_test, mini_test.shape

(array([[  2.,  -2.,  -5.],
        [  1.,  -1., -10.],
        [  3.,  -3.,  -9.],
        [  4.,  -4., -12.]], dtype=float32), (4, 3))

In [77]:
mini_var = F.expand_dims(mini_test.swapaxes(0,1), axis=0)

In [78]:
mini_var.shape

(1, 3, 4)

In [79]:
mini_k = 3; mini_pad = mini_k // 2
minic1 = L.ConvolutionND(ndim=1, in_channels=3, out_channels=5, ksize=2, stride=1, pad=1).to_gpu()

In [83]:
cnn_out_size(i=4, k=2, p=1)

cnn out = 5


In [84]:
print(minic1.W.shape)
print(minic1.W.data)

(5, 3, 2)
[[[ 0.18660849 -0.03519266]
  [-0.3809438  -0.05283932]
  [-0.15335082 -0.06658217]]

 [[ 0.11922466  0.33550972]
  [-0.05792021 -0.04456493]
  [-0.9256264  -1.0981313 ]]

 [[-0.53644639 -0.42481604]
  [-0.17755114  0.88369918]
  [ 0.23915586  0.03831255]]

 [[-0.02067855 -0.08494393]
  [-0.23605981  0.50130999]
  [-0.46147168 -0.84219682]]

 [[ 0.04642114 -0.38426223]
  [ 0.00247118 -0.37618864]
  [-0.2522822   0.42013261]]]


In [95]:
one = minic1.W.data[0]
one.shape

(3, 2)

In [86]:
mini_test[0], mini_test[1]

(array([ 2., -2., -5.], dtype=float32),
 array([  1.,  -1., -10.], dtype=float32))

In [89]:
h1 = one[0][0]*mini_test[0][0]
h2 = one[1][0]*mini_test[0][1]
h3 = one[2][0]*mini_test[0][2]
h4 = one[0][1]*mini_test[1][0]
h5 = one[1][1]*mini_test[1][1]
h6 = one[2][1]*mini_test[1][2]

In [93]:
sum([h1, h2, h3, h4, h5, h6])

array(2.5853271484375, dtype=float32)

In [94]:
mini_cnn_out = minic1(mini_var).data

In [92]:
mini_cnn_out.shape, mini_cnn_out

((1, 5, 5), array([[[  0.36820415,   2.58532691,   2.75324011,   3.95238686,
            4.11041927],
         [  6.25080585,  16.34380913,  20.45681572,  23.55994606,
           11.81609631],
         [ -2.80859303,  -3.60521054,  -7.02081251,  -8.92289925,
           -4.30545139],
         [  3.03847623,  10.57383537,  10.65110779,  12.5607357 ,
            6.39918518],
         [ -2.11681032,  -2.86008883,  -1.23864245,  -2.67149591,
            3.20318627]]], dtype=float32))

In [37]:
F.max_pooling_nd(mini_cnn_out, ksize=2, stride=1, pad=mini_pad).data

/afs/inf.ed.ac.uk/group/project/lowres/work/anaconda3/envs/mtenv/lib/python3.6/site-packages/chainer/utils/experimental.py:104: FutureWarning: chainer.functions.pooling.MaxPoolingND is experimental. The interface can change in the future.
  FutureWarning)


array([[[ 0.50540251,  0.70818657,  0.9858464 ,  0.9858464 ,  0.68322808],
        [-0.81462538, -0.81462538, -2.12726021, -2.12726021, -2.71596646]]], dtype=float32)